In [42]:
with open('data.txt') as f:
    lines= f.readlines()
    lines= [ line.rstrip() for line in lines]
    
lines    

['hindi movies for adults',
 'are panda dogs real',
 'asuedraw winning numbers',
 'sentry replacement keys',
 'rebuilding nicad battery packs',
 'can you overdose on melatonin',
 'list of browsers',
 'que son numeros naturales',
 'mini motos honda',
 'mulatto',
 'gene autry museum',
 'dominicks',
 'wordz',
 'all super bowl winners',
 'townhouse with garage floor plan',
 'teacher appreciation quotes',
 'outdoor wooden steps',
 'ceramic classes',
 'boletos primera plus',
 'how to build a shower pan',
 'vintage shoe company',
 'pent house magazine',
 'ejemplos de quimica',
 'www carnival cruise lines',
 'where is mica found',
 'sonhar com carrapatos',
 'hopi tribe history',
 'vehiculos siniestrados',
 'what causes cheese to mold',
 'how to write a quotation letter',
 'creperas',
 'left side rib pain',
 'safe search off',
 'vtu bangalore results',
 'segunda mano herramientas',
 'concurso magisterio 2013',
 'catherine bell measurements',
 'primera gerra mundial',
 'eduardo costa ouvir todas

In [53]:
posts = nltk.corpus.nps_chat.xml_posts()[:10000]

def dialogue_act_features(post):
    features = {}
    for word in nltk.word_tokenize(post):
        features['contains({})'.format(word.lower())] = True
    return features

featuresets = [(dialogue_act_features(post.text), post.get('class')) for post in posts]

# 10% of the total data
size = int(len(featuresets) * 0.1)

# first 10% for test_set to check the accuracy, and rest 90% after the first 10% for training
train_set, test_set = featuresets[size:], featuresets[:size]

# get the classifer from the training set
classifier = nltk.NaiveBayesClassifier.train(train_set)
# to check the accuracy - 0.67
# print(nltk.classify.accuracy(classifier, test_set))

question_types = ["whQuestion","ynQuestion"]
def is_ques_using_nltk(ques):
    question_type = classifier.classify(dialogue_act_features(ques)) 
    return question_type in question_types

In [55]:
question_pattern = ["do i", "do you", "what", "who", "is it", "why","would you", "how","is there",
                    "are there", "is it so", "is this true" ,"to know", "is that true", "are we", "am i", 
                   "question is", "tell me more", "can i", "can we", "tell me", "can you explain",
                   "question","answer", "questions", "answers", "ask"]

helping_verbs = ["is","am","can", "are", "do", "does"]
# check with custom pipeline if still this is a question mark it as a question
def is_question(question):
    question = question.lower().strip()
    if not is_ques_using_nltk(question):
        is_ques = False
        # check if any of pattern exist in sentence
        for pattern in question_pattern:
            is_ques  = pattern in question
            if is_ques:
                break

        # there could be multiple sentences so divide the sentence
        sentence_arr = question.split(".")
        for sentence in sentence_arr:
            if len(sentence.strip()):
                # if question ends with ? or start with any helping verb
                # word_tokenize will strip by default
                first_word = nltk.word_tokenize(sentence)[0]
                if sentence.endswith("?") or first_word in helping_verbs:
                    is_ques = True
                    break
        return is_ques    
    else:
        return True

In [60]:
List_of_question_and_ans =[]


for line in lines:
    t= is_question(line)
    List_of_question_and_ans.append(t)


In [65]:
data_set = pd.DataFrame(list(zip(lines, List_of_question_and_ans)), columns=['Question', "Yes/No"])
data_set

,Question,Yes/No
0,hindi movies for adults,False
1,are panda dogs real,True
2,asuedraw winning numbers,False
3,sentry replacement keys,False
4,rebuilding nicad battery packs,False
...,...,...
9988,sjogren syndrome pictures,False
9989,how do you renew a passport,True
9990,medellin dim,False
9991,concursos funcab,False


In [66]:
data= data_set.to_csv('answes.csv')